In [1]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from scipy.stats import ttest_ind, levene, norm
import pingouin as pg
from sklearn.linear_model import LassoCV
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import label_binarize
from bayes_opt import BayesianOptimization

# 分类器
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing,tree,model_selection,svm,naive_bayes
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance
from lce import LCEClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.multiclass import OneVsRestClassifier

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics 
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, mean_squared_error, confusion_matrix
from math import sqrt
import csv
import math
import warnings

KeyboardInterrupt: 

In [ ]:
print(plt.style.available)
plt.style.use('seaborn-white')
plt.figure(dpi=300)
# plt.xkcd() #手绘风格
# with plt.style.context(('dark_background')) #限制在某一代码块

In [ ]:
warnings.filterwarnings('ignore')
xgb.set_config(verbosity=0)

In [ ]:
xlsx_a = 'E:/featureExtraction/Analysis/小波+Log/results/1.xlsx'
xlsx_b = 'E:/featureExtraction/Analysis/小波+Log/results/2.xlsx'
xlsx_c = 'E:/featureExtraction/Analysis/小波+Log/results/3.xlsx'
data_a = pd.read_excel(xlsx_a)
data_b = pd.read_excel(xlsx_b)
data_c = pd.read_excel(xlsx_c)
print(data_a.shape,data_b.shape,data_c.shape)

In [ ]:
rows_a,cols_a = data_a.shape
rows_b,cols_b = data_b.shape
rows_c,cols_c = data_c.shape
labels_a = np.zeros(rows_a)
labels_b = np.ones(rows_b)
labels_c = np.full(rows_c,2)
data_a.insert(0, '对应结局', labels_a)
data_b.insert(0, '对应结局', labels_b)
data_c.insert(0, '对应结局', labels_c)
data = pd.concat([data_a,data_b,data_c])
print(data.shape)

In [ ]:
data_train, data_test = train_test_split(data,test_size=0.25,random_state = 121)
data_train_a = data_train[:][data_train['对应结局'] == 0]
data_train_b = data_train[:][data_train['对应结局'] == 1]
data_train_c = data_train[:][data_train['对应结局'] == 2]
data_test_a = data_test[:][data_test['对应结局'] == 0]
data_test_b = data_test[:][data_test['对应结局'] == 1]
data_test_c = data_test[:][data_test['对应结局'] == 2]
print(data_train_a.shape)
print(data_train_b.shape)
print(data_train_c.shape)
print(data_test_a.shape)
print(data_test_b.shape)
print(data_test_c.shape)
data_train_a.to_csv("训练集1.csv",index=False,sep=',')
data_train_b.to_csv("训练集2.csv",index=False,sep=',')
data_train_c.to_csv("训练集3.csv",index=False,sep=',')
data_test_a.to_csv("测试集1.csv",index=False,sep=',')
data_test_b.to_csv("测试集2.csv",index=False,sep=',')
data_test_c.to_csv("测试集3.csv",index=False,sep=',')

In [ ]:
df = pd.concat([data_train_a,data_train_b,data_train_c])
df

In [ ]:
index = []
for colName in data.columns[:]:
    if levene(data_train_a[colName], data_train_b[colName], data_train_c[colName])[1] > 0.05: 
        aov = pg.anova(data=df, 
               dv=colName,  #因变量
               between='对应结局', 
               detailed=False)
        if aov['p-unc'].item() < 0.05: 
            index.append(colName)
    else: 
        aov = pg.welch_anova(data=df, 
               dv=colName,  #因变量
               between='对应结局'
               )
        if aov['p-unc'].item() < 0.05: 
            index.append(colName)
print(len(index))
print(index)

In [ ]:
index.insert(0, '对应结局')
print(index)

In [ ]:
data_train_a = data_train_a[index]
data_train_b = data_train_b[index]
data_train_c = data_train_c[index]
data_train = pd.concat([data_train_a, data_train_b, data_train_c])
data_train.index = range(len(data_train))
X_train = data_train[data_train.columns[1:]]
# 注意下面两行在训练集与测试集上的区别
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_train = pd.DataFrame(X_train)
X_train.columns = index[1:]
y_train = data_train['对应结局']

In [ ]:
data_test_a = data_test_a[index]
data_test_b = data_test_b[index]
data_test_c = data_test_c[index]
data_test = pd.concat([data_test_a, data_test_b, data_test_c])
data_test.index = range(len(data_test))
X_test = data_test[data_test.columns[1:]]
X_test = scaler.transform(X_test)# 只能用训练集的标准化方法定义
X_test = pd.DataFrame(X_test)
X_test.columns = index[1:]
y_test = data_test['对应结局']

In [ ]:
pearson_corr = data[[c for c in X_train.columns if c not in '对应结局']].corr('pearson')
pp = sns.clustermap(pearson_corr, linewidths=.5, figsize=(50.0, 40.0), cmap='YlGnBu')
plt.setp(pp.ax_heatmap.get_yticklabels(), rotation=0)
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类聚类分析.svg", dpi=300,format="svg",bbox_inches = 'tight')

In [ ]:
def hight_corr(x):
    data_corr = x.corr()>0.8
    a=[]
    
    for i in data_corr.columns:
        if data_corr[i].sum()>=2:
            a.append(i)
 
    return a
print(hight_corr(X_train))
print(len(hight_corr(X_train)))

In [ ]:
index2 = set(index)-set(hight_corr(X_train))
index2 = list(index2)
index2.remove("对应结局")
print(index2)
print(len(index2))

In [ ]:
X_train = X_train[index2]
X_test = X_test[index2]
X_train.describe()

In [ ]:
alphas = np.logspace(-4,1,100)
model_lassoCV = LassoCV(alphas = alphas, max_iter = 1e8, cv = 10).fit(X_train,y_train)
coef = pd.Series(model_lassoCV.coef_, index = X_train.columns)
print(model_lassoCV.alpha_)
print('%s %d'%('Lasso picked',sum(abs(coef)!=0)))
print(coef[abs(coef)!=0])
index = coef[abs(coef)!=0].index

In [ ]:
coefficient=model_lassoCV.coef_[abs(model_lassoCV.coef_)!=0]
coefficient
model_lassoCV.intercept_

In [ ]:
font2 = {'family': 'Arial',
     'weight': 'normal',
     'size': 16, }
Features1 = ('original_shape_Flatness','original_ngtdm_Contrast')
Coefficient1= [0.101894, 0.023968]
Features2 = ('wavelet-LHL_glcm_Imc2','wavelet-LHL_gldm_DependenceVariance')
Coefficient2= [-0.017605, -0.087181]
Features3 = ('wavelet-LLL_gldm_DependenceEntropy')
Coefficient3= [0.094268]
Features4 = ('log-sigma-3-mm-3D_glszm_SmallAreaEmphasis')
Coefficient4= [-0.033748]
plt.bar(Features1, Coefficient1, color='#ff073a')  # 横放条形图函数 barh
plt.bar(Features2, Coefficient2, color='#1f77b4')
plt.bar(Features3, Coefficient3, color='#ff073a')  
plt.bar(Features4, Coefficient4, color='#1f77b4')
# 添加竖线
# plt.axvline(x=0, linestyle='-', color='black')
plt.xticks(fontsize=14,family='Arial', rotation=45,ha = 'right',va = 'top')
plt.yticks(fontsize=14,family='Arial')
plt.ylabel('Lasso regression coefficient',font2)
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类特征权重图.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
plt.figure(figsize=(6,5), dpi= 300)
corr = X_train.corr()
mask = np.zeros_like(corr)
print(mask)
mask[np.triu_indices_from(mask,k=1)] = True   # 抽取上三角矩阵
print(mask)
sns.heatmap(corr                           #计算特征间的相关性
            , xticklabels=X_train.corr().columns
            , yticklabels=X_train.corr().columns
            , cmap='YlGnBu'
            , center=0.35
            , annot=True
            , annot_kws={"size": 14}
            , mask=mask
            , square=True)

plt.title('Correlogram of features', fontsize=18, family='Arial')
plt.xticks(fontsize=14, family='Arial',rotation=45,ha = 'right',va = 'top')
plt.yticks(fontsize=14, family='Arial')
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类热图.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
MSEs = model_lassoCV.mse_path_
"""
MSEs_mean, MSEs_std = [], []
for i in range(len(MSEs)):
    MSEs_mean.append(MSEs[i].mean())
    MSEs_std.append(MSEs[i].std())
"""
MSEs_mean = np.apply_along_axis(np.mean,1,MSEs)
MSEs_std = np.apply_along_axis(np.std,1,MSEs)

plt.figure()
plt.errorbar(model_lassoCV.alphas_,MSEs_mean    #x, y数据，一一对应
             , yerr=MSEs_std                    #y误差范围
             , fmt="o"                          #数据点标记
             , ms=3                             #数据点大小
             , mfc="r"                          #数据点颜色
             , mec="r"                          #数据点边缘颜色
             , ecolor="lightblue"               #误差棒颜色
             , elinewidth=1                     #误差棒线宽
             , capsize=4                        #误差棒边界线长度
             , capthick=0.5)                      #误差棒边界线厚度
plt.semilogx()
plt.axvline(model_lassoCV.alpha_,color = 'black',ls="--")
plt.xlabel('Lambda',font2)
plt.ylabel('MSE',font2)
plt.xticks(fontsize=14,family='Arial')
plt.yticks(fontsize=14,family='Arial')
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类Lasso选值图.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
coefs = model_lassoCV.path(X_train_raw,y_train,alphas = alphas, max_iter = 1e7)[1].T
plt.figure()
plt.semilogx(model_lassoCV.alphas_,coefs, '-')
plt.axvline(model_lassoCV.alpha_,color = 'black',ls="--")
plt.xlabel('Lambda',font2)
plt.ylabel('Coefficients',font2)
plt.xticks(fontsize=14,family='Arial')
plt.yticks(fontsize=14,family='Arial')
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类Lasso迭代图.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
index1 = ['对应结局','original_shape_Flatness','original_ngtdm_Contrast','wavelet-LHL_glcm_Imc2','wavelet-LHL_gldm_DependenceVariance',
         'wavelet-LLL_gldm_DependenceEntropy','log-sigma-3-mm-3D_glszm_SmallAreaEmphasis']
index2 = ['对应结局','has-miR371a-3P','has-miR-375-5P']
index3 = ['对应结局','has-miR371a-3P']
index4 = ['对应结局','has-miR-375-5P']
index5 = ['original_shape_Flatness','original_ngtdm_Contrast','wavelet-LHL_glcm_Imc2','wavelet-LHL_gldm_DependenceVariance',
         'wavelet-LLL_gldm_DependenceEntropy','log-sigma-3-mm-3D_glszm_SmallAreaEmphasis','has-miR371a-3P','has-miR-375-5P']

In [ ]:
data_train_a1 = data_train_a[index1]
data_train_b1 = data_train_b[index1]
data_train_c1 = data_train_c[index1]
data_train1 = pd.concat([data_train_a1, data_train_b1, data_train_c1])
X_train1 = data_train1[data_train1.columns[1:]]
# 注意下面两行在训练集与测试集上的区别
scaler = StandardScaler()
scaler.fit(X_train1)
X_train1 = scaler.transform(X_train1)
X_train1 = pd.DataFrame(X_train1)
X_train1.columns = data_train1.columns[1:]
y_train1 = data_train1['对应结局']

In [ ]:
data_test_a1 = data_test_a[index1]
data_test_b1 = data_test_b[index1]
data_test_c1 = data_test_c[index1]
data_test1 = pd.concat([data_test_a1, data_test_b1, data_test_c1])
X_test1 = data_test1[data_test1.columns[1:]]
X_test1 = scaler.transform(X_test1)# 只能用训练集的标准化方法定义
X_test1 = pd.DataFrame(X_test1)
X_test1.columns = data_test1.columns[1:]
y_test1 = data_test1['对应结局']

In [ ]:
data_train_a2 = data_train_a[index2]
data_train_b2 = data_train_b[index2]
data_train_c2 = data_train_c[index2]
data_train2 = pd.concat([data_train_a2, data_train_b2, data_train_c2])
X_train2 = data_train2[data_train2.columns[1:]]
# 注意下面两行在训练集与测试集上的区别
scaler = StandardScaler()
scaler.fit(X_train2)
X_train2 = scaler.transform(X_train2)
X_train2 = pd.DataFrame(X_train2)
X_train2.columns = data_train2.columns[1:]
y_train2 = data_train2['对应结局']

In [ ]:
data_test_a2 = data_test_a[index2]
data_test_b2 = data_test_b[index2]
data_test_c2 = data_test_c[index2]
data_test2 = pd.concat([data_test_a2, data_test_b2, data_test_c2])
X_test2 = data_test2[data_test2.columns[1:]]
X_test2 = scaler.transform(X_test2)# 只能用训练集的标准化方法定义
X_test2 = pd.DataFrame(X_test2)
X_test2.columns = data_test2.columns[1:]
y_test2 = data_test2['对应结局']

In [ ]:
data_train_a3 = data_train_a[index3]
data_train_b3 = data_train_b[index3]
data_train_c3 = data_train_c[index3]
data_train3 = pd.concat([data_train_a3, data_train_b3, data_train_c3])
X_train3 = data_train3[data_train3.columns[1:]]
# 注意下面两行在训练集与测试集上的区别
scaler = StandardScaler()
scaler.fit(X_train3)
X_train3 = scaler.transform(X_train3)
X_train3 = pd.DataFrame(X_train3)
X_train3.columns = data_train3.columns[1:]
y_train3 = data_train3['对应结局']

In [ ]:
data_test_a3 = data_test_a[index3]
data_test_b3 = data_test_b[index3]
data_test_c3 = data_test_c[index3]
data_test3 = pd.concat([data_test_a3, data_test_b3, data_test_c3])
X_test3 = data_test3[data_test3.columns[1:]]
X_test3 = scaler.transform(X_test3)# 只能用训练集的标准化方法定义
X_test3 = pd.DataFrame(X_test3)
X_test3.columns = data_test3.columns[1:]
y_test3 = data_test3['对应结局']

In [ ]:
data_train_a4 = data_train_a[index4]
data_train_b4 = data_train_b[index4]
data_train_c4 = data_train_c[index4]
data_train4 = pd.concat([data_train_a4, data_train_b4, data_train_c4])
X_train4 = data_train4[data_train4.columns[1:]]
# 注意下面两行在训练集与测试集上的区别
scaler = StandardScaler()
scaler.fit(X_train4)
X_train4 = scaler.transform(X_train4)
X_train4 = pd.DataFrame(X_train4)
X_train4.columns = data_train4.columns[1:]
y_train4 = data_train4['对应结局']

In [ ]:
data_test_a4 = data_test_a[index4]
data_test_b4 = data_test_b[index4]
data_test_c4 = data_test_c[index4]
data_test4 = pd.concat([data_test_a4, data_test_b4, data_test_c4])
X_test4 = data_test4[data_test4.columns[1:]]
X_test4 = scaler.transform(X_test4)# 只能用训练集的标准化方法定义
X_test4 = pd.DataFrame(X_test4)
X_test4.columns = data_test4.columns[1:]
y_test4 = data_test4['对应结局']

In [ ]:
data_train_a5 = data_train_a[index5]
data_train_b5 = data_train_b[index5]
data_train_c5 = data_train_c[index5]
data_train5 = pd.concat([data_train_a5, data_train_b5, data_train_c5])
X_train5 = data_train5[data_train5.columns[:]]
# 注意下面两行在训练集与测试集上的区别
scaler = StandardScaler()
scaler.fit(X_train5)
X_train5 = scaler.transform(X_train5)
X_train5 = pd.DataFrame(X_train5)
X_train5.columns = data_train5.columns[:]

In [ ]:
def AUC_CI(auc, label, alpha = 0.05):
    label = np.array(label)#防止label不是array类型
    n1, n2 = np.sum(label == 1), np.sum(label == 0)
    q1 = auc / (2-auc)
    q2 = (2 * auc ** 2) / (1 + auc)
    se = np.sqrt((auc * (1 - auc) + (n1 - 1) * (q1 - auc ** 2) + (n2 -1) * (q2 - auc ** 2)) / (n1 * n2))
    confidence_level = 1 - alpha
    z_lower, z_upper = norm.interval(confidence_level)
    lowerb, upperb = auc + z_lower * se, auc + z_upper * se
    if upperb > 1:
        upperb = 1.000
    return (format(lowerb, '.3f'), format(upperb, '.3f'))

In [ ]:
def c_matrix(y_true, y_pred):
    C = confusion_matrix(y_true, y_pred)
    plt.matshow(C, cmap=plt.cm.Blues) # 根据最下面的图按自己需求更改颜色
    plt.colorbar()
    thresh = C.max() / 2.
    for i in range(len(C)):
        for j in range(len(C)):
            plt.annotate(C[j, i], xy=(i, j), horizontalalignment='center', verticalalignment='center', color="white" if C[j, i] > thresh else "black")
    # plt.tick_params(labelsize=15) # 设置左边和上面的label类别如0,1,2,3,4的字体大小。
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # plt.ylabel('True label', fontdict={'f00amily': 'Times New Roman', 'size': 20}) # 设置字体大小。
    # plt.xlabel('Predicted label', fontdict={'family': 'Times New Roman', 'size': 20})
    plt.xticks([0,1,2], labels=['N','T','V']) # 将x轴或y轴坐标，刻度 替换为文字/字符
    plt.yticks([0,1,2], labels=['N','T','V'], rotation='vertical')
    plt.show()

In [ ]:
# ROC曲线绘制
def ROC(y_train1, y_train_probs):
    font = {'family': 'Arial',
                'size': 12,
                }
    sns.set(font_scale=1.2)
    plt.rc('font',family='Arial')
    plt.style.use('seaborn-white')
    plt.figure(dpi=300)

    global fprmacro, tprmacro, CI_train4
    y_train = label_binarize(y_train1, classes=[0, 1, 2])
    fpr1, tpr1, thresholds1 = roc_curve(y_train[:,0], y_train_probs[:,0], pos_label = 1)
    fpr2, tpr2, thresholds2 = roc_curve(y_train[:,1], y_train_probs[:,1], pos_label = 1)
    fpr3, tpr3, thresholds3 = roc_curve(y_train[:,2], y_train_probs[:,2], pos_label = 1)

    all_fpr = np.unique(np.concatenate([fpr1, fpr2, fpr3]))
    mean_tpr = np.zeros_like(all_fpr)
    mean_tpr += np.interp(all_fpr, fpr1, tpr1)
    mean_tpr += np.interp(all_fpr, fpr2, tpr2)
    mean_tpr += np.interp(all_fpr, fpr3, tpr3)
    mean_tpr /= 3
    fprmacro = all_fpr
    tprmacro = mean_tpr
    fprmacro = np.insert(fprmacro,0,values=[0])
    tprmacro = np.insert(tprmacro,0,values=[0])
    
    y1 = tpr1 - fpr1
    y2 = tpr2 - fpr2
    y3 = tpr3 - fpr3
    y4 = tprmacro - fprmacro
    Youden_index1 = np.argmax(y1)# Only the first occurrence is returned.
    Youden_index2 = np.argmax(y2)
    Youden_index3 = np.argmax(y3)
    Youden_index4 = np.argmax(y4)
    optimal_threshold = np.array([thresholds1[Youden_index1],thresholds2[Youden_index2],thresholds3[Youden_index3]])
    point = np.array([[1-fpr1[Youden_index1], tpr1[Youden_index1]],[1-fpr2[Youden_index2], tpr2[Youden_index2]],[1-fpr3[Youden_index3], tpr3[Youden_index3]],[1-fprmacro[Youden_index4], tprmacro[Youden_index4]]])
    print(optimal_threshold)
    print('Specificity and Sensitivity')
    print(point)
    
    plt.figure()
    # plt.plot(fpr, tpr, marker = 'o')
    auc1 = roc_auc_score(y_train[:,0], y_train_probs[:,0])
    auc2 = roc_auc_score(y_train[:,1], y_train_probs[:,1])
    auc3 = roc_auc_score(y_train[:,2], y_train_probs[:,2])
    auc4 = metrics.roc_auc_score(y_train1, y_train_probs, average='macro', multi_class='ovr')
    CI_train1=AUC_CI(auc1, y_train[:,0], alpha = 0.05)
    CI_train2=AUC_CI(auc2, y_train[:,1], alpha = 0.05)
    CI_train3=AUC_CI(auc3, y_train[:,2], alpha = 0.05)
    CI_train4=[0,0]
    CI_train4[0]=(float(CI_train1[0])+float(CI_train2[0])+float(CI_train3[0]))/3
    CI_train4[1]=(float(CI_train1[1])+float(CI_train2[1])+float(CI_train3[1]))/3

    plt.plot(fpr1,tpr1,'#1f77b4', label='{}= {} ({}-{})'.format('N vs.O_AUC', '%.3f' % auc1, CI_train1[0], CI_train1[1]))
    plt.plot(fpr2,tpr2,'#ff7f0e', label='{}= {} ({}-{})'.format('T vs.O_AUC', '%.3f' % auc2, CI_train2[0], CI_train2[1]))
    plt.plot(fpr3,tpr3,'#2ca02c', label='{}= {} ({}-{})'.format('V vs.O_AUC', '%.3f' % auc3, CI_train3[0], CI_train3[1]))
    plt.plot(fprmacro, tprmacro, label='{}= {} ({}-{})'.format('macro-average ROC_AUC', '%.3f' % auc4, '%.3f' % CI_train4[0], '%.3f' % CI_train4[1]),
             color='navy', linestyle=':', linewidth=2)

    plt.xlabel("1 - Specificity")
    plt.ylabel("Sensitivity")
    plt.legend(loc='lower right',fontsize = 10)
    plt.plot([0,1],[0,1],'k--')
    plt.show()
    return fprmacro, tprmacro, CI_train4

In [ ]:
coefficient=[0.10189258,  0.02396634,  -0.01760259,  -0.08718162, 0.09426855, -0.03374816]
model_lassoCV_intercept_=0.8148148148148141
print('{}= {}{}{}*{}{}{}*{}{}{}*{}{}{}*{}{}{}*{}{}{}*{}'.format('Radscore', '%.8f' % model_lassoCV_intercept_,'+' if coefficient[0]>=0 else '',
                                                                 '%.8f' % coefficient[0],X_train1.columns[0],'+'if coefficient[1]>=0 else '',
                                                                 '%.8f' % coefficient[1],X_train1.columns[1],'+'if coefficient[2]>=0 else '',
                                                                 '%.8f' % coefficient[2],X_train1.columns[2],'+'if coefficient[3]>=0 else '',
                                                                 '%.8f' % coefficient[3],X_train1.columns[3],'+'if coefficient[4]>=0 else '',
                                                                 '%.8f' % coefficient[4],X_train1.columns[4],'+'if coefficient[5]>=0 else '',
                                                                 '%.8f' % coefficient[5],X_train1.columns[5]))

In [ ]:
def f(x,y):
    return model_lassoCV_intercept_+coefficient[0]*x.iloc[y,0]+coefficient[1]*x.iloc[y,1]+coefficient[2]*x.iloc[y,2]+coefficient[3]*x.iloc[y,3]+coefficient[4]*x.iloc[y,4]+coefficient[5]*x.iloc[y,5]
train_score = []
for i in range(len(y_train1)):
    a = f(X_train1,i)
    train_score.append(a)
dfs = pd.DataFrame(columns = ["label", "Radscore"])
dfs['label'] = list(y_train1)
dfs['Radscore'] = train_score
dfs.to_csv("train-score_lasso三分类.csv",index=False,sep=',')

test_score = []
for i in range(len(y_test1)):
    a = f(X_test1,i)
    test_score.append(a)
dfs = pd.DataFrame(columns = ["label", "Radscore"])
dfs['label'] = list(y_test1)
dfs['Radscore'] = test_score
dfs.to_csv("test-score_lasso三分类.csv",index=False,sep=',')

In [ ]:
model_LR = OneVsRestClassifier(LogisticRegression(random_state = 100))
model_LR.fit(X_train1,y_train1)

predicted = model_LR.predict(X_train1)
y_train_probs4 = model_LR.predict_proba(X_train1)
auc4 = metrics.roc_auc_score(y_train1, y_train_probs4, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_train1, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_train1, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_train1, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_train1, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc4 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_train1, predicted, digits=3))

c_matrix(y_train1, predicted)

ROC(y_train1, y_train_probs4)
fprmacro_train4=fprmacro
tprmacro_train4=tprmacro
CI_macro_train4=[0,0]
CI_macro_train4[0]=CI_train4[0]
CI_macro_train4[1]=CI_train4[1]

predicted = model_LR.predict(X_test1)  # 测试样本预测
y_test_probs4 = model_LR.predict_proba(X_test1)

# 输出测试集预测评价指标结果
auc_score4 = metrics.roc_auc_score(y_test1, y_test_probs4, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_test1, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_test1, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_test1, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_test1, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc_score4 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_test1, predicted, digits=3))

c_matrix(y_test1, predicted)

ROC(y_test1, y_test_probs4)
fprmacro_test4=fprmacro
tprmacro_test4=tprmacro
CI_macro_test4=[0,0]
CI_macro_test4[0]=CI_train4[0]
CI_macro_test4[1]=CI_train4[1]

In [ ]:
model_SVM = OneVsRestClassifier(svm.SVC(kernel='rbf',gamma = 'scale',probability=True, random_state=100))
model_SVM.fit(X_train1,y_train1)

predicted = model_SVM.predict(X_train1)
y_train_probs5 = model_SVM.predict_proba(X_train1)
auc5 = metrics.roc_auc_score(y_train1, y_train_probs5, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_train1, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_train1, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_train1, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_train1, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc5 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_train1, predicted, digits=3))

c_matrix(y_train1, predicted)

ROC(y_train1, y_train_probs5)
fprmacro_train5=fprmacro
tprmacro_train5=tprmacro
CI_macro_train5=[0,0]
CI_macro_train5[0]=CI_train4[0]
CI_macro_train5[1]=CI_train4[1]

predicted = model_SVM.predict(X_test1)  # 测试样本预测
y_test_probs5 = model_SVM.predict_proba(X_test1)

# 输出测试集预测评价指标结果
auc_score5 = metrics.roc_auc_score(y_test1, y_test_probs5, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_test1, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_test1, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_test1, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_test1, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc_score5 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_test1, predicted, digits=3))

c_matrix(y_test1, predicted)

ROC(y_test1, y_test_probs5)
fprmacro_test5=fprmacro
tprmacro_test5=tprmacro
CI_macro_test5=[0,0]
CI_macro_test5[0]=CI_train4[0]
CI_macro_test5[1]=CI_train4[1]

In [ ]:
model_RF = OneVsRestClassifier(RandomForestClassifier(n_estimators = 100, random_state = 100, class_weight = 'balanced', criterion = 'entropy'))
model_RF.fit(X_train1,y_train1)

predicted = model_RF.predict(X_train1)
y_train_probs6 = model_RF.predict_proba(X_train1)
auc6 = metrics.roc_auc_score(y_train1, y_train_probs6, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_train1, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_train1, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_train1, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_train1, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc6 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_train1, predicted, digits=3))

c_matrix(y_train1, predicted)

ROC(y_train1, y_train_probs6)
fprmacro_train6=fprmacro
tprmacro_train6=tprmacro
CI_macro_train6=[0,0]
CI_macro_train6[0]=CI_train4[0]
CI_macro_train6[1]=CI_train4[1]

predicted = model_RF.predict(X_test1)  # 测试样本预测
y_test_probs6 = model_RF.predict_proba(X_test1)

# 输出测试集预测评价指标结果
auc_score6 = metrics.roc_auc_score(y_test1, y_test_probs6, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_test1, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_test1, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_test1, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_test1, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc_score6 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_test1, predicted, digits=3))

c_matrix(y_test1, predicted)

ROC(y_test1, y_test_probs6)
fprmacro_test6=fprmacro
tprmacro_test6=tprmacro
CI_macro_test6=[0,0]
CI_macro_test6[0]=CI_train4[0]
CI_macro_test6[1]=CI_train4[1]

In [ ]:
model_ADA = OneVsRestClassifier(AdaBoostClassifier(n_estimators=50, random_state = 100))
model_ADA.fit(X_train1,y_train1)

predicted = model_ADA.predict(X_train1)
y_train_probs7 = model_ADA.predict_proba(X_train1)
auc7 = metrics.roc_auc_score(y_train1, y_train_probs7, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_train1, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_train1, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_train1, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_train1, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc7 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_train1, predicted, digits=3))

c_matrix(y_train1, predicted)

ROC(y_train1, y_train_probs7)
fprmacro_train7=fprmacro
tprmacro_train7=tprmacro
CI_macro_train7=[0,0]
CI_macro_train7[0]=CI_train4[0]
CI_macro_train7[1]=CI_train4[1]

predicted = model_ADA.predict(X_test1)  # 测试样本预测
y_test_probs7 = model_ADA.predict_proba(X_test1)

# 输出测试集预测评价指标结果
auc_score7 = metrics.roc_auc_score(y_test1, y_test_probs7, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_test1, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_test1, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_test1, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_test1, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc_score7 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_test1, predicted, digits=3))

c_matrix(y_test1, predicted)

ROC(y_test1, y_test_probs7)
fprmacro_test7=fprmacro
tprmacro_test7=tprmacro
CI_macro_test7=[0,0]
CI_macro_test7[0]=CI_train4[0]
CI_macro_test7[1]=CI_train4[1]

In [ ]:
model_GBDT = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=50, learning_rate=1.0,max_depth=1, random_state=100))
model_GBDT.fit(X_train1,y_train1)

predicted = model_GBDT.predict(X_train1)
y_train_probs8 = model_GBDT.predict_proba(X_train1)
auc8 = metrics.roc_auc_score(y_train1, y_train_probs8, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_train1, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_train1, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_train1, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_train1, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc8 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_train1, predicted, digits=3))

c_matrix(y_train1, predicted)

ROC(y_train1, y_train_probs8)
fprmacro_train8=fprmacro
tprmacro_train8=tprmacro
CI_macro_train8=[0,0]
CI_macro_train8[0]=CI_train4[0]
CI_macro_train8[1]=CI_train4[1]

predicted = model_GBDT.predict(X_test1)  # 测试样本预测
y_test_probs8 = model_GBDT.predict_proba(X_test1)

# 输出测试集预测评价指标结果
auc_score8 = metrics.roc_auc_score(y_test1, y_test_probs8, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_test1, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_test1, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_test1, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_test1, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc_score8 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_test1, predicted, digits=3))

c_matrix(y_test1, predicted)

ROC(y_test1, y_test_probs8)
fprmacro_test8=fprmacro
tprmacro_test8=tprmacro
CI_macro_test8=[0,0]
CI_macro_test8[0]=CI_train4[0]
CI_macro_test8[1]=CI_train4[1]

dataframe = pd.DataFrame({'label':y_train1,'pred1':y_train_probs8[:,0],'pred2':y_train_probs8[:,1],'pred3':y_train_probs8[:,2]})
dataframe.to_csv("train-CT-三分类.csv",index=False,sep=',')
dataframe = pd.DataFrame({'label':y_test1,'pred1':y_test_probs8[:,0],'pred2':y_test_probs8[:,1],'pred3':y_test_probs8[:,2]})
dataframe.to_csv("test-CT-三分类.csv",index=False,sep=',')

In [ ]:
model_XGB = xgb.XGBClassifier()
params = {
    'max_depth': 6,# 构建树的深度，越大越容易过拟合
    'learning_rate': 0.3,# 如同学习率  
    'n_estimators': 100, #树的个数
    'silent': 0,#设置成1则没有运行信息输出，最好是设置为0.是否在运行升级时打印消息
    'objective': 'multi:softprob',#多分类的问题 指定学习任务和相应的学习目标
    'num_class': 3, # 类别数，多分类与 multisoftmax 并用
    'nthread': 4,# cpu 线程数 默认最大
    'gamma': 0,# 树的叶子节点上作进一步分区所需的最小损失减少,越大越保守，一般0.1、0.2这样子。
    'subsample': 1,# 随机采样训练样本 训练实例的子采样比
    'min_child_weight': 1, 
                          # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
                          #，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
                          #这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting
    'colsample_bytree': 1,
    'max_delta_step': 0,#最大增量步长，我们允许每个树的权重估计。
    'reg_lambda': 1,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'seed': 100,#随机种子
    'reg_alpha': 0, # L1 正则项参数
    #'scale_pos_weight': 1, #如果取值大于0的话，在类别样本不平衡的情况下有助于快速收敛。平衡正负权重
    'eval_metric': 'auc'
}
model_XGB.fit(X_train1, y_train1)

predicted = model_XGB.predict(X_train1)
y_train_probs1 = model_XGB.predict_proba(X_train1)
auc1 = metrics.roc_auc_score(y_train1, y_train_probs1, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_train1, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_train1, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_train1, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_train1, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc1 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_train1, predicted, digits=3))

c_matrix(y_train1, predicted)

ROC(y_train1, y_train_probs1)
fprmacro_train1=fprmacro
tprmacro_train1=tprmacro
CI_macro_train1=[0,0]
CI_macro_train1[0]=CI_train4[0]
CI_macro_train1[1]=CI_train4[1]

predicted = model_XGB.predict(X_test1)  # 测试样本预测
y_test_probs1 = model_XGB.predict_proba(X_test1)

# 输出测试集预测评价指标结果
auc_score1 = metrics.roc_auc_score(y_test1, y_test_probs1, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_test1, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_test1, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_test1, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_test1, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc_score1 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_test1, predicted, digits=3))

c_matrix(y_test1, predicted)

ROC(y_test1, y_test_probs1)
fprmacro_test1=fprmacro
tprmacro_test1=tprmacro
CI_macro_test1=[0,0]
CI_macro_test1[0]=CI_train4[0]
CI_macro_test1[1]=CI_train4[1]

# 显示重要特征
plot_importance(model_XGB)
plt.show()

# 打印测试集RMSE
rmse = sqrt(mean_squared_error(np.array(list(y_test1)), np.array(list(predicted))))
print("rmse:",rmse)

In [ ]:
sns.set(font_scale=1.2)
plt.style.use('seaborn-white')
plt.figure(dpi=300)

plt.plot(fprmacro_train4,tprmacro_train4,'lime', label='{}= {} ({}-{})'.format('LR_macro_AUC', '%.3f' % auc4, '%.3f' % CI_macro_train4[0], '%.3f' % CI_macro_train4[1]))
plt.plot(fprmacro_train5,tprmacro_train5,'magenta', label='{}= {} ({}-{})'.format('SVM_macro_AUC', '%.3f' % auc5, '%.3f' % CI_macro_train5[0], '%.3f' % CI_macro_train5[1]))
plt.plot(fprmacro_train6,tprmacro_train6,'purple', label='{}= {} ({}-{})'.format('RF_macro_AUC', '%.3f' % auc6, '%.3f' % CI_macro_train6[0], '%.3f' % CI_macro_train6[1]))
plt.plot(fprmacro_train7,tprmacro_train7,'coral', label='{}= {} ({}-{})'.format('ADA_macro_AUC', '%.3f' % auc7, '%.3f' % CI_macro_train7[0], '%.3f' % CI_macro_train7[1]))
plt.plot(fprmacro_train8,tprmacro_train8,'yellow', label='{}= {} ({}-{})'.format('GBDT_macro_AUC', '%.3f' % auc8, '%.3f' % CI_macro_train8[0], '%.3f' % CI_macro_train8[1]))
plt.plot(fprmacro_train1,tprmacro_train1,'blue', label='{}= {} ({}-{})'.format('XGB_macro_AUC', '%.3f' % auc1, '%.3f' % CI_macro_train1[0], '%.3f' % CI_macro_train1[1]))

plt.legend(loc='lower right',fontsize = 10)
plt.plot([0,1],[0,1],'k--')
plt.ylabel('Sensitivity',fontsize = 14)
plt.xlabel('1 - Specificity',fontsize = 14)
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类CT1.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
sns.set(font_scale=1.2)
plt.style.use('seaborn-white')
plt.figure(dpi=300)

plt.plot(fprmacro_test4,tprmacro_test4,'lime', label='{}= {} ({}-{})'.format('LR_macro_AUC', '%.3f' % auc_score4, '%.3f' % CI_macro_test4[0], '%.3f' % CI_macro_test4[1]))
plt.plot(fprmacro_test5,tprmacro_test5,'magenta', label='{}= {} ({}-{})'.format('SVM_macro_AUC', '%.3f' % auc_score5, '%.3f' % CI_macro_test5[0], '%.3f' % CI_macro_test5[1]))
plt.plot(fprmacro_test6,tprmacro_test6,'purple', label='{}= {} ({}-{})'.format('RF_macro_AUC', '%.3f' % auc_score6, '%.3f' % CI_macro_test6[0], '%.3f' % CI_macro_test6[1]))
plt.plot(fprmacro_test7,tprmacro_test7,'coral', label='{}= {} ({}-{})'.format('ADA_macro_AUC', '%.3f' % auc_score7, '%.3f' % CI_macro_test7[0], '%.3f' % CI_macro_test7[1]))
plt.plot(fprmacro_test8,tprmacro_test8,'yellow', label='{}= {} ({}-{})'.format('GBDT_macro_AUC', '%.3f' % auc_score8, '%.3f' % CI_macro_test8[0], '%.3f' % CI_macro_test8[1]))
plt.plot(fprmacro_test1,tprmacro_test1,'blue', label='{}= {} ({}-{})'.format('XGB_macro_AUC', '%.3f' % auc_score1, '%.3f' % CI_macro_test1[0], '%.3f' % CI_macro_test1[1]))

plt.legend(loc='lower right',fontsize = 10)
plt.plot([0,1],[0,1],'k--')
plt.ylabel('Sensitivity',fontsize = 14)
plt.xlabel('1 - Specificity',fontsize = 14)
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类CT2.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
#坐标轴负号的处理
plt.rcParams['axes.unicode_minus']=False
# 读取数据
tips = pd.read_csv('C:/Users/Ding/Desktop/融合/三分类融合/训练集miRNA.csv')
# 绘制分组小提琴图
sns.boxplot(x = "label", # 指定x轴的数据
               y = "has-miR371a-3P", # 指定y轴的数据
               data = tips, # 指定绘图的数据集
               order = ['Necrosis/fibrosis','Teratoma','Viable GCT'], # 指定x轴刻度标签的顺序
               palette = 'Set2', # 指定不同性别对应的颜色（因为hue参数为设置为性别变量）
               saturation=1, #饱和度
               whis=1,
               #flierprops = {'marker': 'D'},  # 异常值形状 'markerfacecolor': 'red',  # 形状填充色
               fliersize=5, 
               whiskerprops={'linestyle':'-', 'color':'black'},  # 设置上下须属性
               showmeans=True,  # 箱图显示均值，
               meanprops={'marker': 's','markerfacecolor':'red', 'markeredgecolor':'red','fillstyle':'none','markersize': 5}  # 设置均值属性
              )

plt.xticks(ha = 'center',va = 'top',size=15)
plt.yscale("log")
plt.ylim(0.005,8000)
plt.ylabel('miR371a-3P relative expression',size=15)
plt.xlabel('')
x1, x2, x3= 0, 1, 2
y,h = tips["has-miR371a-3P"].mean()+1600,1000
#绘制横线位置
plt.plot([x2+0.05, x2+0.05, x3, x3], [y-1100, y-900, y-900, y-1100], lw=1, c="k") 
plt.plot([x1, x1, x2-0.05, x2-0.05], [y-1100, y-900, y-900, y-1100], lw=1, c="k")
plt.plot([x1, x1, x3, x3], [y, y+h, y+h, y], lw=1, c="k") 
#添加P值
plt.text((x2+x3)*.5+0.025, y-1200, "*", ha='center', va='bottom', color="k", size=24)
plt.text((x1+x2)*.5-0.025, y-850, "ns", ha='center', va='bottom', color="k", size=15)
plt.text((x1+x3)*.5, y, "*", ha='center', va='bottom', color="k", size=24)
# sns.swarmplot(x='group1', y='rad',data=tips,color='w', alpha=0.8, size=3)
# 显示图形
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/miRNA对比371-1.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
#坐标轴负号的处理
plt.rcParams['axes.unicode_minus']=False
# 读取数据
tips = pd.read_csv('C:/Users/Ding/Desktop/融合/三分类融合/训练集miRNA.csv')
# 绘制分组小提琴图
sns.boxplot(x = "label", # 指定x轴的数据
               y = "has-miR-375-5P", # 指定y轴的数据
               data = tips, # 指定绘图的数据集
               order = ['Necrosis/fibrosis','Teratoma','Viable GCT'], # 指定x轴刻度标签的顺序
               palette = 'Set2', # 指定不同性别对应的颜色（因为hue参数为设置为性别变量）
               saturation=1, #饱和度
               whis=5,
               #flierprops = {'marker': 'D'},  # 异常值形状 'markerfacecolor': 'red',  # 形状填充色
               fliersize=5, 
               whiskerprops={'linestyle':'-', 'color':'black'},  # 设置上下须属性
               showmeans=True,  # 箱图显示均值，
               meanprops={'marker': 's','markerfacecolor':'red', 'markeredgecolor':'red','fillstyle':'none','markersize': 5}  # 设置均值属性
              )

plt.xticks(ha = 'center',va = 'top',size=15)
plt.yscale("log")
plt.ylim(0.005,8000)
plt.ylabel('miR375-5P relative expression',size=15)
plt.xlabel('')
x1, x2, x3= 0, 1, 2
y,h = tips["has-miR-375-5P"].mean()+1600,1000
#绘制横线位置
plt.plot([x2+0.05, x2+0.05, x3, x3], [y-1100, y-900, y-900, y-1100], lw=1, c="k") 
plt.plot([x1, x1, x2-0.05, x2-0.05], [y-1100, y-900, y-900, y-1100], lw=1, c="k")
plt.plot([x1, x1, x3, x3], [y, y+h, y+h, y], lw=1, c="k") 
#添加P值
plt.text((x2+x3)*.5+0.025, y-1200, "*", ha='center', va='bottom', color="k", size=24)
plt.text((x1+x2)*.5-0.025, y-1200, "*", ha='center', va='bottom', color="k", size=24)
plt.text((x1+x3)*.5, y+1000, "ns", ha='center', va='bottom', color="k", size=15)
# sns.swarmplot(x='group1', y='rad',data=tips,color='w', alpha=0.8, size=3)
# 显示图形
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/miRNA对比375-1.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
model_LR = OneVsRestClassifier(LogisticRegression(random_state = 100))
model_LR.fit(X_train2,y_train2)

predicted = model_LR.predict(X_train2)
y_train_probs9 = model_LR.predict_proba(X_train2)
auc9 = metrics.roc_auc_score(y_train2, y_train_probs9, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_train2, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_train2, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_train2, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_train2, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc9 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_train2, predicted, digits=3))

c_matrix(y_train2, predicted)

ROC(y_train2, y_train_probs9)
fprmacro_train9=fprmacro
tprmacro_train9=tprmacro
CI_macro_train9=[0,0]
CI_macro_train9[0]=CI_train4[0]
CI_macro_train9[1]=CI_train4[1]

predicted = model_LR.predict(X_test2)  # 测试样本预测
y_test_probs9 = model_LR.predict_proba(X_test2)

# 输出测试集预测评价指标结果
auc_score9 = metrics.roc_auc_score(y_test2, y_test_probs9, average='macro', multi_class='ovr')  # auc roc曲线下面积
accuracy = metrics.accuracy_score(y_test2, predicted)  # accuracy准确率
Recall = metrics.recall_score(y_test2, predicted, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_test2, predicted, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_test2, predicted, average='weighted') # f1 score f1分数
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("auc: %.2f%%" % (auc_score9 * 100.0))
print("Recall: %.2f%%" % (Recall * 100.0))
print("Prec: %.2f%%" % (Prec * 100.0))
print("F1: %.2f%%" % (f1 * 100.0))
print(classification_report(y_test2, predicted, digits=3))

c_matrix(y_test2, predicted)

ROC(y_test2, y_test_probs9)
fprmacro_test9=fprmacro
tprmacro_test9=tprmacro
CI_macro_test9=[0,0]
CI_macro_test9[0]=CI_train4[0]
CI_macro_test9[1]=CI_train4[1]
dataframe = pd.DataFrame({'label':y_train2,'pred1':y_train_probs9[:,0],'pred2':y_train_probs9[:,1],'pred3':y_train_probs9[:,2]})
dataframe.to_csv("train-RNA-三分类.csv",index=False,sep=',')
dataframe = pd.DataFrame({'label':y_test2,'pred1':y_test_probs9[:,0],'pred2':y_test_probs9[:,1],'pred3':y_test_probs9[:,2]})
dataframe.to_csv("test-RNA-三分类.csv",index=False,sep=',')

In [ ]:
xlsx_a = 'C:/Users/Ding/Desktop/融合/三分类融合/融合train结果.xlsx'
xlsx_b = 'C:/Users/Ding/Desktop/融合/三分类融合/融合test结果.xlsx'
data_a = pd.read_excel(xlsx_a)
data_b = pd.read_excel(xlsx_b)
rows_a,cols_a = data_a.shape
rows_b,cols_b = data_b.shape
y_train_probs3 = np.ones((rows_a,cols_a))
y_train_probs3= data_a.iloc[:,1:4]
y_train3 = data_a['label']
predicted1 = data_a['predicted1']
y_test_probs3 = np.ones((rows_b,cols_b))
y_test_probs3= data_b.iloc[:,1:4]
y_test3 = data_b['label']
predicted2 = data_b['predicted1']
y_train_probs3=y_train_probs3.values
y_test_probs3=y_test_probs3.values

y_train_probs4 = np.ones((rows_a,cols_a))
y_train_probs4= data_a.iloc[:,5:8]
y_train4 = data_a['label']
predicted3 = data_a['predicted2']
y_test_probs4 = np.ones((rows_b,cols_b))
y_test_probs4= data_b.iloc[:,5:8]
y_test4 = data_b['label']
predicted4 = data_b['predicted2']
y_train_probs4=y_train_probs4.values
y_test_probs4=y_test_probs4.values

y_train_probs5 = np.ones((rows_a,cols_a))
y_train_probs5= data_a.iloc[:,9:12]
y_train5 = data_a['label']
predicted5 = data_a['predicted3']
y_test_probs5 = np.ones((rows_b,cols_b))
y_test_probs5= data_b.iloc[:,9:12]
y_test5 = data_b['label']
predicted6 = data_b['predicted3']
y_train_probs5=y_train_probs5.values
y_test_probs5=y_test_probs5.values

In [ ]:
sns.set(font_scale=1.2)
plt.style.use('seaborn-white')
plt.figure(dpi=300)

y_train = label_binarize(y_train5, classes=[0, 1, 2])
fpr1, tpr1, thresholds1 = roc_curve(y_train[:,0], y_train_probs5[:,0], pos_label = 1)
fpr2, tpr2, thresholds2 = roc_curve(y_train[:,1], y_train_probs5[:,1], pos_label = 1)
fpr3, tpr3, thresholds3 = roc_curve(y_train[:,2], y_train_probs5[:,2], pos_label = 1)

all_fpr = np.unique(np.concatenate([fpr1, fpr2, fpr3]))
mean_tpr = np.zeros_like(all_fpr)
mean_tpr += np.interp(all_fpr, fpr1, tpr1)
mean_tpr += np.interp(all_fpr, fpr2, tpr2)
mean_tpr += np.interp(all_fpr, fpr3, tpr3)
mean_tpr /= 3
fprmacro = all_fpr
tprmacro = mean_tpr
fprmacro = np.insert(fprmacro,0,values=[0])
tprmacro = np.insert(tprmacro,0,values=[0])
y1 = tpr1 - fpr1
y2 = tpr2 - fpr2
y3 = tpr3 - fpr3
y4 = tprmacro - fprmacro
Youden_index1 = np.argmax(y1)# Only the first occurrence is returned.
Youden_index2 = np.argmax(y2)
Youden_index3 = np.argmax(y3)
Youden_index4 = np.argmax(y4)
optimal_threshold = np.array([thresholds1[Youden_index1],thresholds2[Youden_index2],thresholds3[Youden_index3]])
point = np.array([[1-fpr1[Youden_index1], tpr1[Youden_index1]],[1-fpr2[Youden_index2], tpr2[Youden_index2]],[1-fpr3[Youden_index3], tpr3[Youden_index3]],[1-fprmacro[Youden_index4], tprmacro[Youden_index4]]])
print(optimal_threshold)
print('Specificity and Sensitivity')
print(point)

plt.figure()
# plt.plot(fpr, tpr, marker = 'o')
auc1 = roc_auc_score(y_train[:,0], y_train_probs5[:,0])
auc2 = roc_auc_score(y_train[:,1], y_train_probs5[:,1])
auc3 = roc_auc_score(y_train[:,2], y_train_probs5[:,2])
auc4 = metrics.roc_auc_score(y_train, y_train_probs5, average='macro', multi_class='ovr')
CI_train1=AUC_CI(auc1, y_train[:,0], alpha = 0.05)
CI_train2=AUC_CI(auc2, y_train[:,1], alpha = 0.05)
CI_train3=AUC_CI(auc3, y_train[:,2], alpha = 0.05)
CI_train4=[0,0]
CI_train4[0]=(float(CI_train1[0])+float(CI_train2[0])+float(CI_train3[0]))/3
CI_train4[1]=(float(CI_train1[1])+float(CI_train2[1])+float(CI_train3[1]))/3

plt.plot(fpr1,tpr1,'#1f77b4', label='{}= {} ({}-{})'.format('N vs.O_AUC', '%.3f' % auc1, CI_train1[0], CI_train1[1]))
plt.plot(fpr2,tpr2,'#ff7f0e', label='{}= {} ({}-{})'.format('T vs.O_AUC', '%.3f' % auc2, CI_train2[0], CI_train2[1]))
plt.plot(fpr3,tpr3,'#2ca02c', label='{}= {} ({}-{})'.format('V vs.O_AUC', '%.3f' % auc3, CI_train3[0], CI_train3[1]))
plt.plot(fprmacro, tprmacro, label='{}= {} ({}-{})'.format('macro-average ROC_AUC', '%.3f' % auc4, '%.3f' % CI_train4[0], '%.3f' % CI_train4[1]),
         color='navy', linestyle=':', linewidth=2)

plt.xlabel("1 - Specificity",size='18')
plt.ylabel("Sensitivity",size='18')
plt.xticks(fontsize=16,family='Arial')
plt.yticks(fontsize=16,family='Arial')
plt.legend(loc='lower right',fontsize = 10)
plt.plot([0,1],[0,1],'k--')
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类CT+371+375后融合1.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

fprmacro_train5=fprmacro
tprmacro_train5=tprmacro
CI_macro_train5=[0,0]
CI_macro_train5[0]=float(CI_train4[0])
CI_macro_train5[1]=float(CI_train4[1])

accuracy = metrics.accuracy_score(y_train5, predicted5)  # accuracy准确率
Recall = metrics.recall_score(y_train5, predicted5, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_train5, predicted5, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_train5, predicted5, average='weighted') # f1 score f1分数
print("Accuracy: %.3f%%" % (accuracy * 100.0))
print("Recall: %.3f%%" % (Recall * 100.0))
print("Prec: %.3f%%" % (Prec * 100.0))
print("F1: %.3f%%" % (f1 * 100.0))
print(classification_report(y_train5, predicted5, digits=3))

C = confusion_matrix(y_train5, predicted5)
plt.matshow(C, cmap=plt.cm.Blues) # 根据最下面的图按自己需求更改颜色
plt.colorbar()
thresh = C.max() / 2.
for i in range(len(C)):
    for j in range(len(C)):
        plt.annotate(C[j, i], xy=(i, j), horizontalalignment='center', verticalalignment='center', color="white" if C[j, i] > thresh else "black")
plt.tick_params(labelsize=18) # 设置左边和上面的label类别如0,1,2,3,4的字体大小。
plt.ylabel('True label', fontdict={'family': 'Arial', 'size': 20}) # 设置字体大小
plt.xlabel('Predicted label', fontdict={'family': 'Arial', 'size': 20})
plt.xticks([0,1,2], labels=['N','T','V']) # 将x轴或y轴坐标，刻度 替换为文字/字符
plt.yticks([0,1,2], labels=['N','T','V'], rotation='vertical')
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类CT+371+375后融合混淆矩阵1.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
sns.set(font_scale=1.2)
plt.style.use('seaborn-white')
plt.figure(dpi=300)

y_test = label_binarize(y_test5, classes=[0, 1, 2])
fpr1, tpr1, thresholds1 = roc_curve(y_test[:,0], y_test_probs5[:,0], pos_label = 1)
fpr2, tpr2, thresholds2 = roc_curve(y_test[:,1], y_test_probs5[:,1], pos_label = 1)
fpr3, tpr3, thresholds3 = roc_curve(y_test[:,2], y_test_probs5[:,2], pos_label = 1)

all_fpr = np.unique(np.concatenate([fpr1, fpr2, fpr3]))
mean_tpr = np.zeros_like(all_fpr)
mean_tpr += np.interp(all_fpr, fpr1, tpr1)
mean_tpr += np.interp(all_fpr, fpr2, tpr2)
mean_tpr += np.interp(all_fpr, fpr3, tpr3)
mean_tpr /= 3
fprmacro = all_fpr
tprmacro = mean_tpr
fprmacro = np.insert(fprmacro,0,values=[0])
tprmacro = np.insert(tprmacro,0,values=[0])
y1 = tpr1 - fpr1
y2 = tpr2 - fpr2
y3 = tpr3 - fpr3
y4 = tprmacro - fprmacro
Youden_index1 = np.argmax(y1)# Only the first occurrence is returned.
Youden_index2 = np.argmax(y2)
Youden_index3 = np.argmax(y3)
Youden_index4 = np.argmax(y4)
optimal_threshold = np.array([thresholds1[Youden_index1],thresholds2[Youden_index2],thresholds3[Youden_index3]])
point = np.array([[1-fpr1[Youden_index1], tpr1[Youden_index1]],[1-fpr2[Youden_index2], tpr2[Youden_index2]],[1-fpr3[Youden_index3], tpr3[Youden_index3]],[1-fprmacro[Youden_index4], tprmacro[Youden_index4]]])
print(optimal_threshold)
print('Specificity and Sensitivity')
print(point)

plt.figure()
# plt.plot(fpr, tpr, marker = 'o')
auc_score1 = roc_auc_score(y_test[:,0], y_test_probs5[:,0])
auc_score2 = roc_auc_score(y_test[:,1], y_test_probs5[:,1])
auc_score3 = roc_auc_score(y_test[:,2], y_test_probs5[:,2])
auc_score4 = metrics.roc_auc_score(y_test, y_test_probs5, average='macro', multi_class='ovr')
CI_test1=AUC_CI(auc_score1, y_test[:,0], alpha = 0.05)
CI_test2=AUC_CI(auc_score2, y_test[:,1], alpha = 0.05)
CI_test3=AUC_CI(auc_score3, y_test[:,2], alpha = 0.05)
CI_test4=[0,0]
CI_test4[0]=(float(CI_test1[0])+float(CI_test2[0])+float(CI_test3[0]))/3
CI_test4[1]=(float(CI_test1[1])+float(CI_test2[1])+float(CI_test3[1]))/3

plt.plot(fpr1,tpr1,'#1f77b4', label='{}= {} ({}-{})'.format('N vs.O_AUC', '%.3f' % auc_score1, CI_test1[0], CI_test1[1]))
plt.plot(fpr2,tpr2,'#ff7f0e', label='{}= {} ({}-{})'.format('T vs.O_AUC', '%.3f' % auc_score2, CI_test2[0], CI_test2[1]))
plt.plot(fpr3,tpr3,'#2ca02c', label='{}= {} ({}-{})'.format('V vs.O_AUC', '%.3f' % auc_score3, CI_test3[0], CI_test3[1]))
plt.plot(fprmacro, tprmacro, label='{}= {} ({}-{})'.format('macro-average ROC_AUC', '%.3f' % auc_score4, '%.3f' % CI_test4[0], '%.3f' % CI_test4[1]),
         color='navy', linestyle=':', linewidth=2)

plt.xlabel("1 - Specificity",size='18')
plt.ylabel("Sensitivity",size='18')
plt.xticks(fontsize=16,family='Arial')
plt.yticks(fontsize=16,family='Arial')
plt.legend(loc='lower right',fontsize = 10)
plt.plot([0,1],[0,1],'k--')
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类CT+371+375后融合2.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

fprmacro_test5=fprmacro
tprmacro_test5=tprmacro
CI_macro_test5=[0,0]
CI_macro_test5[0]=float(CI_test4[0])
CI_macro_test5[1]=float(CI_test4[1])

accuracy = metrics.accuracy_score(y_test5, predicted6)  # accuracy准确率
Recall = metrics.recall_score(y_test5, predicted6, average='weighted')   # recall 召回率
Prec = metrics.precision_score(y_test5, predicted6, average='weighted')  #precision 精度
f1 = metrics.f1_score(y_test5, predicted6, average='weighted') # f1 score f1分数
print("Accuracy: %.3f%%" % (accuracy * 100.0))
print("Recall: %.3f%%" % (Recall * 100.0))
print("Prec: %.3f%%" % (Prec * 100.0))
print("F1: %.3f%%" % (f1 * 100.0))
print(classification_report(y_test5, predicted6, digits=3))

C = confusion_matrix(y_test5, predicted6)
plt.matshow(C, cmap=plt.cm.Blues) # 根据最下面的图按自己需求更改颜色
plt.colorbar()
thresh = C.max() / 2.
for i in range(len(C)):
    for j in range(len(C)):
        plt.annotate(C[j, i], xy=(i, j), horizontalalignment='center', verticalalignment='center', color="white" if C[j, i] > thresh else "black")
plt.tick_params(labelsize=18) # 设置左边和上面的label类别如0,1,2,3,4的字体大小。
plt.ylabel('True label', fontdict={'family': 'Arial', 'size': 20}) # 设置字体大小
plt.xlabel('Predicted label', fontdict={'family': 'Arial', 'size': 20})
plt.xticks([0,1,2], labels=['N','T','V']) # 将x轴或y轴坐标，刻度 替换为文字/字符
plt.yticks([0,1,2], labels=['N','T','V'], rotation='vertical')
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类CT+371+375后融合混淆矩阵2.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
import scipy.stats as st

class DelongTest():
    def __init__(self,preds1,preds2,label,threshold=0.05):
        '''
        preds1:the output of model1
        preds2:the output of model2
        label :the actual label
        '''
        self._preds1=preds1
        self._preds2=preds2
        self._label=label
        self.threshold=threshold
        self._show_result()

    def _auc(self,X, Y)->float:
        return 1/(len(X)*len(Y)) * sum([self._kernel(x, y) for x in X for y in Y])

    def _kernel(self,X, Y)->float:
        '''
        Mann-Whitney statistic
        '''
        return .5 if Y==X else int(Y < X)

    def _structural_components(self,X, Y)->list:
        V10 = [1/len(Y) * sum([self._kernel(x, y) for y in Y]) for x in X]
        V01 = [1/len(X) * sum([self._kernel(x, y) for x in X]) for y in Y]
        return V10, V01

    def _get_S_entry(self,V_A, V_B, auc_A, auc_B)->float:
        return 1/(len(V_A)-1) * sum([(a-auc_A)*(b-auc_B) for a,b in zip(V_A, V_B)])
    
    def _z_score(self,var_A, var_B, covar_AB, auc_A, auc_B):
        return (auc_A - auc_B)/((var_A + var_B - 2*covar_AB )**(.5)+ 1e-8)

    def _group_preds_by_label(self,preds, actual)->list:
        X = [p for (p, a) in zip(preds, actual) if a]
        Y = [p for (p, a) in zip(preds, actual) if not a]
        return X, Y

    def _compute_z_p(self):
        X_A, Y_A = self._group_preds_by_label(self._preds1, self._label)
        X_B, Y_B = self._group_preds_by_label(self._preds2, self._label)

        V_A10, V_A01 = self._structural_components(X_A, Y_A)
        V_B10, V_B01 = self._structural_components(X_B, Y_B)

        auc_A = self._auc(X_A, Y_A)
        auc_B = self._auc(X_B, Y_B)

        # Compute entries of covariance matrix S (covar_AB = covar_BA)
        var_A = (self._get_S_entry(V_A10, V_A10, auc_A, auc_A) * 1/len(V_A10)+ self._get_S_entry(V_A01, V_A01, auc_A, auc_A) * 1/len(V_A01))
        var_B = (self._get_S_entry(V_B10, V_B10, auc_B, auc_B) * 1/len(V_B10)+ self._get_S_entry(V_B01, V_B01, auc_B, auc_B) * 1/len(V_B01))
        covar_AB = (self._get_S_entry(V_A10, V_B10, auc_A, auc_B) * 1/len(V_A10)+ self._get_S_entry(V_A01, V_B01, auc_A, auc_B) * 1/len(V_A01))

        # Two tailed test
        z = self._z_score(var_A, var_B, covar_AB, auc_A, auc_B)
        p = st.norm.sf(abs(z))*2

        return z,p

    def _show_result(self):
        z,p=self._compute_z_p()
        print(f"z score = {z:.5f};\np value = {p:.5f};")
        if p < self.threshold :print("There is a significant difference")
        else:        print("There is NO significant difference")

DelongTest(y_test_probs8[:,0],y_test_probs3[:,0],y_test1)
DelongTest(y_test_probs8[:,0],y_test_probs4[:,0],y_test1)
DelongTest(y_test_probs8[:,0],y_test_probs5[:,0],y_test1)
DelongTest(y_test_probs8[:,1],y_test_probs3[:,1],y_test1)
DelongTest(y_test_probs8[:,1],y_test_probs4[:,1],y_test1)
DelongTest(y_test_probs8[:,1],y_test_probs5[:,1],y_test1)
DelongTest(y_test_probs8[:,2],y_test_probs3[:,2],y_test1)
DelongTest(y_test_probs8[:,2],y_test_probs4[:,2],y_test1)
DelongTest(y_test_probs8[:,2],y_test_probs5[:,2],y_test1)

In [ ]:
y_train5 = label_binarize(y_train5, classes=[0, 1, 2])
y_test5 = label_binarize(y_test5, classes=[0, 1, 2])
auc5 = metrics.roc_auc_score(y_train5, y_train_probs5, average='macro', multi_class='ovr')
auc_score5 = metrics.roc_auc_score(y_test5, y_test_probs5, average='macro', multi_class='ovr')

In [ ]:
sns.set(font_scale=1.2)
plt.style.use('seaborn-white')
plt.figure(dpi=300)

plt.plot(fprmacro_train8,tprmacro_train8,'#1f77b4', label='{}= {} ({}-{})'.format('Radiomic_macro_AUC', '%.3f' % auc8, '%.3f' % CI_macro_train8[0], '%.3f' % CI_macro_train8[1]))
# plt.plot(fprmacro_train3,tprmacro_train3,'#ff7f0e', label='{}= {} ({}-{})'.format('Radiomic+miR371_macro_AUC', '%.3f' % auc3, '%.3f' % CI_macro_train3[0], '%.3f' % CI_macro_train3[1]))
# plt.plot(fprmacro_train4,tprmacro_train4,'#2ca02c', label='{}= {} ({}-{})'.format('Radiomic+miR375_macro_AUC', '%.3f' % auc4, '%.3f' % CI_macro_train4[0], '%.3f' % CI_macro_train4[1]))
plt.plot(fprmacro_train5,tprmacro_train5,'#d62728', label='{}= {} ({}-{})'.format('Radiomic+miR371+miR375_macro_AUC', '%.3f' % auc5, '%.3f' % CI_macro_train5[0], '%.3f' % CI_macro_train5[1]))

plt.legend(loc='lower right',fontsize = 9)
plt.plot([0,1],[0,1],'k--')
plt.ylabel('Sensitivity',fontsize = 14)
plt.xlabel('1 - Specificity',fontsize = 14)
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类后融合1.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
sns.set(font_scale=1.2)
plt.style.use('seaborn-white')
plt.figure(dpi=300)

plt.plot(fprmacro_test8,tprmacro_test8,'#1f77b4', label='{}= {} ({}-{})'.format('Radiomic_macro_AUC', '%.3f' % auc_score8, '%.3f' % CI_macro_test8[0], '%.3f' % CI_macro_test8[1]))
# plt.plot(fprmacro_test3,tprmacro_test3,'#ff7f0e', label='{}= {} ({}-{})'.format('Radiomic+miR371_macro_AUC', '%.3f' % auc_score3, '%.3f' % CI_macro_test3[0], '%.3f' % CI_macro_test3[1]))
# plt.plot(fprmacro_test4,tprmacro_test4,'#2ca02c', label='{}= {} ({}-{})'.format('Radiomic+miR375_macro_AUC', '%.3f' % auc_score4, '%.3f' % CI_macro_test4[0], '%.3f' % CI_macro_test4[1]))
plt.plot(fprmacro_test5,tprmacro_test5,'#d62728', label='{}= {} ({}-{})'.format('Radiomic+miR371+miR375_macro_AUC', '%.3f' % auc_score5, '%.3f' % CI_macro_test5[0], '%.3f' % CI_macro_test5[1]))

plt.legend(loc='lower right',fontsize = 9)
plt.plot([0,1],[0,1],'k--')
plt.ylabel('Sensitivity',fontsize = 14)
plt.xlabel('1 - Specificity',fontsize = 14)
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类后融合2.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
from sklearn.metrics import log_loss
print(log_loss(y_train1,y_train_probs8))
print(log_loss(y_train3,y_train_probs12))
print(log_loss(y_train3,y_train_probs13))
print(log_loss(y_train3,y_train_probs3))
print(log_loss(y_train4,y_train_probs4))
print(log_loss(y_train2,y_train_probs9))
print(log_loss(y_train5,y_train_probs5))

print(log_loss(y_test1,y_test_probs8))
print(log_loss(y_test3,y_test_probs12))
print(log_loss(y_test3,y_test_probs13))
print(log_loss(y_test3,y_test_probs3))
print(log_loss(y_test4,y_test_probs4))
print(log_loss(y_test2,y_test_probs9))
print(log_loss(y_test5,y_test_probs5))

In [ ]:
from sklearn.metrics import brier_score_loss as BS
y_train = label_binarize(y_train1, classes=[0, 1, 2])
BS1_1=BS(y_train[:,0], y_train_probs8[:,0])
BS1_2=BS(y_train[:,1], y_train_probs8[:,1])
BS1_3=BS(y_train[:,2], y_train_probs8[:,2])

BS2_1=BS(y_train[:,0], y_train_probs12[:,0])
BS2_2=BS(y_train[:,1], y_train_probs12[:,1])
BS2_3=BS(y_train[:,2], y_train_probs12[:,2])

BS3_1=BS(y_train[:,0], y_train_probs13[:,0])
BS3_2=BS(y_train[:,1], y_train_probs13[:,1])
BS3_3=BS(y_train[:,2], y_train_probs13[:,2])

BS4_1=BS(y_train[:,0], y_train_probs3[:,0])
BS4_2=BS(y_train[:,1], y_train_probs3[:,1])
BS4_3=BS(y_train[:,2], y_train_probs3[:,2])

BS5_1=BS(y_train[:,0], y_train_probs4[:,0])
BS5_2=BS(y_train[:,1], y_train_probs4[:,1])
BS5_3=BS(y_train[:,2], y_train_probs4[:,2])

BS6_1=BS(y_train[:,0], y_train_probs9[:,0])
BS6_2=BS(y_train[:,1], y_train_probs9[:,1])
BS6_3=BS(y_train[:,2], y_train_probs9[:,2])

BS7_1=BS(y_train[:,0], y_train_probs5[:,0])
BS7_2=BS(y_train[:,1], y_train_probs5[:,1])
BS7_3=BS(y_train[:,2], y_train_probs5[:,2])
print((BS1_1+BS1_2+BS1_3)/3)
print((BS2_1+BS2_2+BS2_3)/3)
print((BS3_1+BS3_2+BS3_3)/3)
print((BS4_1+BS4_2+BS4_3)/3)
print((BS5_1+BS5_2+BS5_3)/3)
print((BS6_1+BS6_2+BS6_3)/3)
print((BS7_1+BS7_2+BS7_3)/3)

In [ ]:
y_test = label_binarize(y_test1, classes=[0, 1, 2])
BS1_1=BS(y_test[:,0], y_test_probs8[:,0])
BS1_2=BS(y_test[:,1], y_test_probs8[:,1])
BS1_3=BS(y_test[:,2], y_test_probs8[:,2])

BS2_1=BS(y_test[:,0], y_test_probs12[:,0])
BS2_2=BS(y_test[:,1], y_test_probs12[:,1])
BS2_3=BS(y_test[:,2], y_test_probs12[:,2])

BS3_1=BS(y_test[:,0], y_test_probs13[:,0])
BS3_2=BS(y_test[:,1], y_test_probs13[:,1])
BS3_3=BS(y_test[:,2], y_test_probs13[:,2])

BS4_1=BS(y_test[:,0], y_test_probs3[:,0])
BS4_2=BS(y_test[:,1], y_test_probs3[:,1])
BS4_3=BS(y_test[:,2], y_test_probs3[:,2])

BS5_1=BS(y_test[:,0], y_test_probs4[:,0])
BS5_2=BS(y_test[:,1], y_test_probs4[:,1])
BS5_3=BS(y_test[:,2], y_test_probs4[:,2])

BS6_1=BS(y_test[:,0], y_test_probs9[:,0])
BS6_2=BS(y_test[:,1], y_test_probs9[:,1])
BS6_3=BS(y_test[:,2], y_test_probs9[:,2])

BS7_1=BS(y_test[:,0], y_test_probs5[:,0])
BS7_2=BS(y_test[:,1], y_test_probs5[:,1])
BS7_3=BS(y_test[:,2], y_test_probs5[:,2])
print((BS1_1+BS1_2+BS1_3)/3)
print((BS2_1+BS2_2+BS2_3)/3)
print((BS3_1+BS3_2+BS3_3)/3)
print((BS4_1+BS4_2+BS4_3)/3)
print((BS5_1+BS5_2+BS5_3)/3)
print((BS6_1+BS6_2+BS6_3)/3)
print((BS7_1+BS7_2+BS7_3)/3)

In [ ]:
from sklearn.calibration import calibration_curve
from sklearn.calibration import CalibratedClassifierCV
# calibrated_clf = CalibratedClassifierCV(model_XGB, cv="prefit", method="sigmoid")
# calibrated_clf.fit(X_train, y_train)
# y_test_probs = calibrated_clf.predict_proba(X_test)

plt.figure(figsize=(7, 7))
dpi=300
ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
y_test1 = label_binarize(y_test1, classes=[0, 1, 2])
y_test3 = label_binarize(y_test3, classes=[0, 1, 2])
# y_test4 = label_binarize(y_test4, classes=[0, 1, 2])
y_test5 = label_binarize(y_test5, classes=[0, 1, 2])

y_test_probs8 = (y_test_probs8 - y_test_probs8.min()) / (y_test_probs8.max() - y_test_probs8.min())
fraction_of_positives, mean_predicted_value = calibration_curve(y_test1[:,1], y_test_probs8[:,1], n_bins=4,strategy='quantile')
ax1.plot(mean_predicted_value, fraction_of_positives, "o-",label="%s" % ("Radiomic", ))

# y_test_probs3 = (y_test_probs3 - y_test_probs3.min()) / (y_test_probs3.max() - y_test_probs3.min())
# fraction_of_positives, mean_predicted_value = calibration_curve(y_test3[:,1], y_test_probs3[:,1], n_bins=3,strategy='quantile')
# ax1.plot(mean_predicted_value, fraction_of_positives, "s-",label="%s" % ("Radiomic+miR371", ))

# y_test_probs4 = (y_test_probs4 - y_test_probs4.min()) / (y_test_probs4.max() - y_test_probs4.min())
# fraction_of_positives, mean_predicted_value = calibration_curve(y_test3[:,1], y_test_probs4[:,1], n_bins=3,strategy='quantile')
# ax1.plot(mean_predicted_value, fraction_of_positives, "^-",label="%s" % ("Radiomic+miR375", ))

y_test_probs5 = (y_test_probs5 - y_test_probs5.min()) / (y_test_probs5.max() - y_test_probs5.min())
fraction_of_positives, mean_predicted_value = calibration_curve(y_test3[:,1], y_test_probs5[:,1], n_bins=4,strategy='quantile')
ax1.plot(mean_predicted_value, fraction_of_positives, "D-",label="%s" % ("Radiomic+miR371+miR375", ))
ax1.set_ylabel("Fraction of positives",size='18')
ax1.set_ylim([-0.05, 1.05])
plt.xticks(size='18')
plt.yticks(size='18')
ax1.set_xlabel("Predicted probability",size='18')
ax1.legend(loc="lower right",fontsize = 14)
ax1.set_title('Calibration plots  (reliability curve)',size='18')
plt.tight_layout()
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类校准曲线.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
from sklearn.calibration import calibration_curve
from sklearn.calibration import CalibratedClassifierCV
# calibrated_clf = CalibratedClassifierCV(model_XGB, cv="prefit", method="sigmoid")
# calibrated_clf.fit(X_train, y_train)
# y_test_probs = calibrated_clf.predict_proba(X_test)

plt.figure(figsize=(7, 7))
dpi=300
ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
y_test1 = label_binarize(y_test1, classes=[0, 1, 2])
# y_test3 = label_binarize(y_test3, classes=[0, 1, 2])
# y_test4 = label_binarize(y_test4, classes=[0, 1, 2])
y_test5 = label_binarize(y_test5, classes=[0, 1, 2])

y_test_probs8 = (y_test_probs8 - y_test_probs8.min()) / (y_test_probs8.max() - y_test_probs8.min())
fraction_of_positives, mean_predicted_value = calibration_curve(y_test1[:,2], y_test_probs8[:,2], n_bins=4,strategy='quantile')
ax1.plot(mean_predicted_value, fraction_of_positives, "o-",label="%s" % ("Radiomic", ))

# y_test_probs3 = (y_test_probs3 - y_test_probs3.min()) / (y_test_probs3.max() - y_test_probs3.min())
# fraction_of_positives, mean_predicted_value = calibration_curve(y_test3[:,2], y_test_probs3[:,2], n_bins=3,strategy='quantile')
# ax1.plot(mean_predicted_value, fraction_of_positives, "s-",label="%s" % ("Radiomic+miR371", ))

# y_test_probs4 = (y_test_probs4 - y_test_probs4.min()) / (y_test_probs4.max() - y_test_probs4.min())
# fraction_of_positives, mean_predicted_value = calibration_curve(y_test3[:,2], y_test_probs4[:,2], n_bins=3,strategy='quantile')
# ax1.plot(mean_predicted_value, fraction_of_positives, "^-",label="%s" % ("Radiomic+miR375", ))

y_test_probs5 = (y_test_probs5 - y_test_probs5.min()) / (y_test_probs5.max() - y_test_probs5.min())
fraction_of_positives, mean_predicted_value = calibration_curve(y_test3[:,2], y_test_probs5[:,2], n_bins=4,strategy='quantile')
ax1.plot(mean_predicted_value, fraction_of_positives, "D-",label="%s" % ("Radiomic+miR371+miR375", ))
ax1.set_ylabel("Fraction of positives",size='18')
ax1.set_ylim([-0.05, 1.05])
plt.xticks(size='18')
plt.yticks(size='18')
ax1.set_xlabel("Predicted probability",size='18')
ax1.legend(loc="lower right",fontsize = 14)
ax1.set_title('Calibration plots  (reliability curve)',size='18')
plt.tight_layout()
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类校准曲线2.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
import numpy as np
from scipy.stats import chi2
import statsmodels.api as sm

# 对预测概率分组
def Hosmer_Lemeshow_test(y, y_pred, num_groups):
    grouped_observed = np.array_split(y, num_groups)  
    grouped_predicted = np.array_split(y_pred, num_groups) 

    # 计算每组的观察值和预测值比例
    observed_ratio = [np.mean(g) for g in grouped_observed]
    predicted_ratio = [np.mean(g) for g in grouped_predicted]

    # 执行Hosmer-Lemeshow检验
    hl_statistic = np.sum(((o - p) ** 2) / (p * (1 - p)) for o, p in zip(observed_ratio, predicted_ratio))
    hl_p_value = chi2.sf(hl_statistic, num_groups - 2)
    print("Hosmer-Lemeshow statistic:", hl_statistic)
    print("p-value:", hl_p_value)
Hosmer_Lemeshow_test(y_train1,y_train_probs8,4)
Hosmer_Lemeshow_test(y_train1,y_train_probs12,4)
Hosmer_Lemeshow_test(y_train1,y_train_probs13,4)
Hosmer_Lemeshow_test(y_train3,y_train_probs3,4)
Hosmer_Lemeshow_test(y_train4,y_train_probs4,4)
Hosmer_Lemeshow_test(y_train1,y_train_probs9,4)
Hosmer_Lemeshow_test(y_train5,y_train_probs5,4)

Hosmer_Lemeshow_test(y_test1,y_test_probs8,4)
Hosmer_Lemeshow_test(y_test1,y_test_probs12,4)
Hosmer_Lemeshow_test(y_test1,y_test_probs13,4)
Hosmer_Lemeshow_test(y_test3,y_test_probs3,4)
Hosmer_Lemeshow_test(y_test4,y_test_probs4,4)
Hosmer_Lemeshow_test(y_test1,y_test_probs9,4)
Hosmer_Lemeshow_test(y_test5,y_test_probs5,4)

In [ ]:
from sklearn.metrics import confusion_matrix


def calculate_net_benefit_model(thresh_group, y_pred_score, y_label):
    net_benefit_model = np.array([])
    for thresh in thresh_group:
        y_pred_label = y_pred_score > thresh
        tn, fp, fn, tp = confusion_matrix(y_label, y_pred_label).ravel()
        n = len(y_label)
        net_benefit = (tp / n) - (fp / n) * (thresh / (1 - thresh))
        net_benefit_model = np.append(net_benefit_model, net_benefit)
    return net_benefit_model


def calculate_net_benefit_all(thresh_group, y_label):
    net_benefit_all = np.array([])
    tn, fp, fn, tp = confusion_matrix(y_label, y_label).ravel()
    total = tp + tn
    for thresh in thresh_group:
        net_benefit = (tp / total) - (tn / total) * (thresh / (1 - thresh))
        net_benefit_all = np.append(net_benefit_all, net_benefit)
    return net_benefit_all


def plot_DCA(ax, thresh_group, net_benefit_model1, net_benefit_model4, net_benefit_all):
    #Plot
    ax.plot(thresh_group, net_benefit_model1, color = '#1f77b4', label = 'Radiomic')
#     ax.plot(thresh_group, net_benefit_model2, color = '#ff7f0e', label = 'Radiomic+miR371')
#     ax.plot(thresh_group, net_benefit_model3, color = '#2ca02c', label = 'Radiomic+miR375')
    ax.plot(thresh_group, net_benefit_model4, color = '#d62728', label = 'Radiomic+miR371+miR375')
    ax.plot(thresh_group, net_benefit_all, color = 'black',label = 'Treat all')
    ax.plot((0, 1), (0, 0), color = 'black', linestyle = ':', label = 'Treat none')

    #Fill，显示出模型较于treat all和treat none好的部分
    y2 = np.maximum(net_benefit_all, 0)
#     y3 = np.maximum(net_benefit_model2, y2)
    y1 = np.maximum(net_benefit_model1, y2)
#     y4 = np.maximum(net_benefit_model3, y2)
    y5 = np.maximum(net_benefit_model4, y2)
    ax.fill_between(thresh_group, y1, y2, color = '#1f77b4', alpha = 0.3)
#     ax.fill_between(thresh_group, y3, y1, color = '#ff7f0e', alpha = 0.3)
#     ax.fill_between(thresh_group, y4, y3, color = '#2ca02c', alpha = 0.3)
    ax.fill_between(thresh_group, y5, y1, color = '#d62728', alpha = 0.3)

    #Figure Configuration， 美化一下细节
    ax.set_xlim(0,0.91)
    ax.set_ylim(net_benefit_model1.min() - 0.1 , net_benefit_model1.max() + 0.1)#adjustify the y axis limitation
    ax.set_xlabel(
        xlabel = 'Threshold Probability', 
        fontdict= {'family': 'Arial', 'fontsize': 18}
        )
    ax.set_ylabel(
        ylabel = 'Net Benefit', 
        fontdict= {'family': 'Arial', 'fontsize': 18}
        )
    plt.xticks(size='18')
    plt.yticks(size='18')
    dpi=300
    ax.grid('major')
    ax.spines['right'].set_color((0.8, 0.8, 0.8))
    ax.spines['top'].set_color((0.8, 0.8, 0.8)) 
    ax.legend(loc = 'upper right',fontsize = 12 )

    return ax


if __name__ == '__main__':
    y_pred_score8 = y_test_probs8[:,1]
    y_label1 = y_test1[:,1]
#     y_pred_score3 = y_test_probs3[:,1]
#     y_label2 = y_test3[:,1]
#     y_pred_score4 = y_test_probs4[:,1]
#     y_label3 = y_test3[:,1]
    y_pred_score5 = y_test_probs5[:,1]
    y_label4 = y_test3[:,1]
    thresh_group = np.arange(0,1,0.05)
    net_benefit_model1 = calculate_net_benefit_model(thresh_group, y_pred_score8, y_label1)
#     net_benefit_model2 = calculate_net_benefit_model(thresh_group, y_pred_score3, y_label2)
#     net_benefit_model3 = calculate_net_benefit_model(thresh_group, y_pred_score4, y_label3)
    net_benefit_model4 = calculate_net_benefit_model(thresh_group, y_pred_score5, y_label4)
    net_benefit_all = calculate_net_benefit_all(thresh_group, y_label1)
    fig, ax = plt.subplots()
    ax = plot_DCA(ax, thresh_group, net_benefit_model1, net_benefit_model4, net_benefit_all)
    ax.set_title('Decision curve',size='18')
    plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类决策曲线.svg", dpi=300,format="svg",bbox_inches = 'tight')
    plt.show()

In [ ]:
def plot_DCA(ax, thresh_group, net_benefit_model1, net_benefit_model4, net_benefit_all):
    #Plot
    ax.plot(thresh_group, net_benefit_model1, color = '#1f77b4', label = 'Radiomic')
#     ax.plot(thresh_group, net_benefit_model2, color = '#ff7f0e', label = 'Radiomic+miR371')
#     ax.plot(thresh_group, net_benefit_model3, color = '#2ca02c', label = 'Radiomic+miR375')
    ax.plot(thresh_group, net_benefit_model4, color = '#d62728', label = 'Radiomic+miR371+miR375')
    ax.plot(thresh_group, net_benefit_all, color = 'black',label = 'Treat all')
    ax.plot((0, 1), (0, 0), color = 'black', linestyle = ':', label = 'Treat none')

    #Fill，显示出模型较于treat all和treat none好的部分
    y2 = np.maximum(net_benefit_all, 0)
#     y3 = np.maximum(net_benefit_model2, y2)
    y1 = np.maximum(net_benefit_model1, y2)
#     y4 = np.maximum(net_benefit_model3, y2)
    y5 = np.maximum(net_benefit_model4, y2)
    ax.fill_between(thresh_group, y1, y2, color = '#1f77b4', alpha = 0.3)
#     ax.fill_between(thresh_group, y4, y1, color = '#ff7f0e', alpha = 0.3)
#     ax.fill_between(thresh_group, y3, y4, color = '#2ca02c', alpha = 0.3)
    ax.fill_between(thresh_group, y5, y1, color = '#d62728', alpha = 0.3)

    #Figure Configuration， 美化一下细节
    ax.set_xlim(0,0.79)
    ax.set_ylim(net_benefit_model1.min() + 0 , net_benefit_model1.max() + 0.1)#adjustify the y axis limitation
    ax.set_xlabel(
        xlabel = 'Threshold Probability', 
        fontdict= {'family': 'Arial', 'fontsize': 18}
        )
    ax.set_ylabel(
        ylabel = 'Net Benefit', 
        fontdict= {'family': 'Arial', 'fontsize': 18}
        )
    plt.xticks(size='18')
    plt.yticks(size='18')
    dpi=300
    ax.grid('major')
    ax.spines['right'].set_color((0.8, 0.8, 0.8))
    ax.spines['top'].set_color((0.8, 0.8, 0.8)) 
    ax.legend(loc = 'upper right',fontsize = 12 )

    return ax

if __name__ == '__main__':
    y_pred_score8 = y_test_probs8[:,2]
    y_label1 = y_test1[:,2]
#     y_pred_score3 = y_test_probs3[:,2]
#     y_label2 = y_test3[:,2]
#     y_pred_score4 = y_test_probs4[:,2]
#     y_label3 = y_test3[:,2]
    y_pred_score5 = y_test_probs5[:,2]
    y_label4 = y_test3[:,2]
    thresh_group = np.arange(0,1,0.05)
    net_benefit_model1 = calculate_net_benefit_model(thresh_group, y_pred_score8, y_label1)
#     net_benefit_model2 = calculate_net_benefit_model(thresh_group, y_pred_score3, y_label2)
#     net_benefit_model3 = calculate_net_benefit_model(thresh_group, y_pred_score4, y_label3)
    net_benefit_model4 = calculate_net_benefit_model(thresh_group, y_pred_score5, y_label4)
    net_benefit_all = calculate_net_benefit_all(thresh_group, y_label1)
    fig, ax = plt.subplots()
    ax = plot_DCA(ax, thresh_group, net_benefit_model1, net_benefit_model4, net_benefit_all)
    ax.set_title('Decision curve',size='18')
    plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类决策曲线2.svg", dpi=300,format="svg",bbox_inches = 'tight')
    plt.show()

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats import norm
import numpy as np
from plotnine import *
"""因变量y必须是int：0或1"""
def logistic_regression(y,x,df):
    model = smf.glm(formula = f'{y} ~ {x}',
                    data = df,
                    family=sm.families.Binomial()).fit()  
    print(model.summary(), end='\n\n\n')
    stat = pd.DataFrame({'p': model.pvalues, # series：每个β的p值
                         # 'beta': model.params, # series：β
                         # 'beta_lower_ci': model.conf_int().iloc[:, 0], #df：β的95%CI
                         # 'beta_upper_ci': model.conf_int().iloc[:, 1], #df：β的95%CI                        
                         'OR': np.exp(model.params), # series：OR
                         'OR_lower_ci': np.exp(model.params - norm.ppf(0.975)*model.bse),
                         'OR_upper_ci': np.exp(model.params + norm.ppf(0.975)*model.bse)}) 
    stat['sig'] = stat.apply(lambda x : "*" if x['p']<0.05 else "no_sig",axis=1)
    stat= stat.sort_values('OR', ascending=True)                            
    print(stat)
    """绘制森林图：OR"""
    forest_df = stat.drop("Intercept")\
                    .reset_index()\
                    .rename(columns={'index': 'independent_var'})\
                    .sort_values('OR', ascending=False)
    print(ggplot(forest_df , aes(y='independent_var', x='OR')) + # y轴（自变量x）为连续变量或二分类变量
          geom_point(aes(color='sig'),size=2) +          
          geom_errorbarh(aes(xmin='OR_lower_ci', 
                             xmax='OR_upper_ci',
                             color ='sig'), height=0.1) +
          scale_color_manual(values = ["red","black"]) +
          scale_y_discrete(limits= forest_df["independent_var"]) +
          guides(color=guide_legend(reverse=True))+
          labs(title='logistic Regression', x='OR', y='variable')+
          geom_vline(xintercept=1, linetype='dashed', color='black')+
          theme_minimal()+
          theme(plot_title=element_text(hjust=0.5)))

In [ ]:
df = pd.read_csv('C:/Users/Ding/Desktop/融合/三分类融合/trainCT+371+375.csv')
logistic_regression(y="N",x="Radscore",df=df)
logistic_regression(y="T",x="Radscore",df=df)
logistic_regression(y="V",x="Radscore",df=df)
logistic_regression(y="N",x="miR371+miR375",df=df)
logistic_regression(y="T",x="miR371+miR375",df=df)
logistic_regression(y="V",x="miR371+miR375",df=df)
logistic_regression(y="N",x="Radscore+miR371",df=df)
logistic_regression(y="T",x="Radscore+miR371",df=df)
logistic_regression(y="V",x="Radscore+miR371",df=df)
logistic_regression(y="N",x="Radscore+miR375",df=df)
logistic_regression(y="T",x="Radscore+miR375",df=df)
logistic_regression(y="V",x="Radscore+miR375",df=df)
logistic_regression(y="N",x="Radscore+miR371+miR375",df=df)
logistic_regression(y="T",x="Radscore+miR371+miR375",df=df)
logistic_regression(y="V",x="Radscore+miR371+miR375",df=df)

In [ ]:
#坐标轴负号的处理
plt.rcParams['axes.unicode_minus']=False
# 读取数据
tips = pd.read_csv('C:/Users/Ding/Desktop/融合/三分类融合/train标准化1.csv')
# 绘制分组小提琴图
sns.boxplot(x = "group1", # 指定x轴的数据
               y = "Texture feature parameter", # 指定y轴的数据
               hue = "group2", # 指定分组变量
               data = tips, # 指定绘图的数据集
               order = ['original_shape_Flatness','original_ngtdm_Contrast','wavelet-LHL_glcm_Imc2','wavelet-LHL_gldm_DependenceVariance',
         'wavelet-LLL_gldm_DependenceEntropy','log-sigma-3-mm-3D_glszm_SmallAreaEmphasis'], # 指定x轴刻度标签的顺序
               palette = 'Set2', # 指定不同性别对应的颜色（因为hue参数为设置为性别变量）
               saturation=1, #饱和度
               whis=1,
               #flierprops = {'marker': 'D'},  # 异常值形状 'markerfacecolor': 'red',  # 形状填充色
               fliersize=4, 
               whiskerprops={'linestyle':'-', 'color':'black'},  # 设置上下须属性
               showmeans=True,  # 箱图显示均值，
               meanprops={'marker': 's','markerfacecolor':'w', 'markeredgecolor':'w','fillstyle':'full','markersize': 5}  # 设置均值属性
              )
# 设置图例
plt.legend(loc = 'upper right', ncol = 1)
plt.xticks(rotation=45, ha = 'right',va = 'top')
plt.xlabel('')
# sns.swarmplot(x='group1', y='rad',data=tips,color='w', alpha=0.8, size=3)
# 显示图形
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类特征对比1.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()

In [ ]:
#坐标轴负号的处理
plt.rcParams['axes.unicode_minus']=False
# 读取数据
tips = pd.read_csv('C:/Users/Ding/Desktop/融合/三分类融合/test标准化1.csv')
# 绘制分组小提琴图
sns.boxplot(x = "group1", # 指定x轴的数据
               y = "Texture feature parameter", # 指定y轴的数据
               hue = "group2", # 指定分组变量
               data = tips, # 指定绘图的数据集
               order = ['original_shape_Flatness','original_ngtdm_Contrast','wavelet-LHL_glcm_Imc2','wavelet-LHL_gldm_DependenceVariance',
         'wavelet-LLL_gldm_DependenceEntropy','log-sigma-3-mm-3D_glszm_SmallAreaEmphasis'], # 指定x轴刻度标签的顺序
               palette = 'Set2', # 指定不同性别对应的颜色（因为hue参数为设置为性别变量）
               saturation=1, #饱和度
               whis=1,
               #flierprops = {'marker': 'D'},  # 异常值形状 'markerfacecolor': 'red',  # 形状填充色
               fliersize=4, 
               whiskerprops={'linestyle':'-', 'color':'black'},  # 设置上下须属性
               showmeans=True,  # 箱图显示均值，
               meanprops={'marker': 's','markerfacecolor':'w', 'markeredgecolor':'w','fillstyle':'full','markersize': 5}  # 设置均值属性
              )
# 设置图例
plt.legend(loc = 'upper right', ncol = 1)
plt.xticks(rotation=45, ha = 'right',va = 'top')
plt.xlabel('')
# sns.swarmplot(x='group1', y='rad',data=tips,color='w', alpha=0.8, size=3)
# 显示图形
plt.savefig("C:/Users/Ding/Desktop/Figures and tables/三分类特征对比2.svg", dpi=300,format="svg",bbox_inches = 'tight')
plt.show()